In [1]:


# =============================================================================
# SETUP
# =============================================================================
import sys
import os
from pathlib import Path

NOTEBOOK_DIR = Path(os.getcwd())
sys.path.insert(0, str(NOTEBOOK_DIR / 'mapk_code_modules'))

print(f"Working directory: {NOTEBOOK_DIR}")
print(f"Added to path:")
print(f"  {NOTEBOOK_DIR / 'mapk_code_modules'}")

# =============================================================================
# IMPORTS
# =============================================================================
import numpy as np
import pandas as pd

# Investigation B modules (UPDATED IMPORTS)
from mapk_module0 import PCOAStructureCache
from mapk_module1 import GroundTruthDataGenerator
from mapk_module2 import CandidateODEFitter  
from mapk_module3 import SCMTrainer
from mapk_module4 import ModelEvaluator
from mapk_runner import InvestigationBReplicateRunner

# ODE system
import mapk_pkn as ode_system

print("\n✓ All modules imported successfully")
print(f"✓ ODE system: {ode_system.__name__}")

Working directory: /Users/ritwikanand/causal_models_dynamical_and_equilibrium/ReCOMB_pCOA_final/mapk_ieg_cascade
Added to path:
  /Users/ritwikanand/causal_models_dynamical_and_equilibrium/ReCOMB_pCOA_final/mapk_ieg_cascade/mapk_code_modules

✓ All modules imported successfully
✓ ODE system: mapk_pkn


In [2]:
runner = InvestigationBReplicateRunner(
    ode_module=ode_system,
    system_name='mapk_ieg_cascade',
    super_cache_dir='mapk_replicate_cache_25'
)

log2_table, fold_table = runner.generate_summary_tables()




GENERATING SUMMARY TABLES FROM CACHE

  Collected 166 affected-only datapoints
  Collected 166 all-variables datapoints

  Applying outlier detection...
    Outliers dropped: 5

LOG2 ERRORS - ALL VARIABLES (mean ± std)
GT→Candidate         ODE         SCM    Baseline  n
       MA→MA 0.509±0.246 0.425±0.200 1.201±0.055 15
       MA→MM 1.921±0.660 0.895±0.081 1.206±0.068 21
     MA→Hill 1.897±0.604 0.896±0.100 1.234±0.117 17
       MM→MA 1.421±0.393 0.753±0.126 0.369±0.020 18
       MM→MM 1.430±0.640 0.103±0.024 0.361±0.003 21
     MM→Hill 1.468±0.625 0.123±0.025 0.361±0.004 16
     Hill→MA 1.256±0.342 0.755±0.088 0.364±0.018 16
     Hill→MM 1.437±0.622 0.103±0.020 0.360±0.006 21
   Hill→Hill 1.454±0.593 0.105±0.022 0.359±0.007 16

FOLD ERRORS - ALL VARIABLES (mean ± std)
GT→Candidate         ODE         SCM    Baseline  n
       MA→MA 1.444±0.266 1.355±0.193 2.301±0.089 15
       MA→MM 4.249±2.466 1.862±0.107 2.309±0.110 21
     MA→Hill 4.081±2.013 1.866±0.134 2.360±0.204 17
       MM→

In [7]:
"""
STANDALONE visualization script 

Generates:
1. PKN (original network)
2. pCOA structures for MA, MM, Hill 
3. MA clean version (all inhibitions: I_MEK, I_ERK, I_CyclinD, but no R variables)

Enhanced version: larger elements, thicker lines, reduced whitespace
"""

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from pathlib import Path
import pickle


# ============================================================================
# PKN
# ============================================================================
G = nx.DiGraph()
G.add_nodes_from(['GF', 'MEK', 'ERK', 'cFos', 'Fra1', 'cJun', 'CyclinD', 'Bcl2'])
G.add_edges_from([
    ('GF', 'MEK'), ('GF', 'ERK'), ('GF', 'cFos'),
    ('MEK', 'ERK'), ('MEK', 'cFos'), ('ERK', 'cFos'),
    ('cFos', 'Fra1'), ('Fra1', 'cJun'), ('cJun', 'cFos'),
    ('cJun', 'CyclinD'), ('cFos', 'Bcl2'),
])

feedback = [('cJun', 'cFos'), ('cFos', 'Fra1'), ('Fra1', 'cJun')]
activation = [e for e in G.edges() if e not in feedback]

fig, ax = plt.subplots(figsize=(16, 8))
pos = {
    'GF': (0, 0), 'MEK': (4, 1.5), 'ERK': (4, -1.5),
    'cFos': (8, 0), 'Fra1': (12, 0), 'cJun': (12, 3.5),
    'CyclinD': (16, 3.5), 'Bcl2': (16, -2),
}

# Enhanced edge drawing - thicker lines, larger arrows
nx.draw_networkx_edges(G, pos, edgelist=activation, edge_color='black', 
                      arrowsize=70, arrowstyle='->', width=8, node_size=14000, ax=ax)
nx.draw_networkx_edges(G, pos, edgelist=feedback, edge_color='#FF9900',
                      arrowsize=70, arrowstyle='->', width=9, node_size=14000, ax=ax)

# Enhanced nodes - larger, thicker borders
nx.draw_networkx_nodes(G, pos, nodelist=['GF'], node_color='#FFD966', 
                      node_shape='s', node_size=14000, edgecolors='black', linewidths=7, ax=ax)
nx.draw_networkx_nodes(G, pos, nodelist=list(set(G.nodes()) - {'GF'}),
                      node_color='#6FA8DC', node_shape='o', node_size=14000,
                      edgecolors='black', linewidths=7, ax=ax)

# Enhanced labels - larger font
nx.draw_networkx_labels(G, pos, font_size=32, font_weight='bold', ax=ax)

# Enhanced legend - sized to avoid overlap
ax.legend(handles=[
    Line2D([0], [0], color='black', linewidth=6, marker='>', markersize=18, label='Activation'),
    Line2D([0], [0], color='#FF9900', linewidth=7, marker='>', markersize=18, label='Feedback cycle'),
], loc='upper left', fontsize=22, frameon=True, fancybox=False,
         edgecolor='black', facecolor='white', framealpha=1.0, borderpad=1.0, labelspacing=0.8)

ax.axis('off')
ax.margins(0.10)  # Slightly increased to prevent node cutoff
plt.tight_layout(pad=0.1)  # Minimal padding
Path('figures').mkdir(exist_ok=True)
plt.savefig('figures/pkn.png', dpi=300, bbox_inches='tight', facecolor='white', pad_inches=0.15)
print("✓ PKN saved")
plt.close()


# ============================================================================
# PCOA STRUCTURES 
# ============================================================================

# Find first replicate with pcoa cache
super_cache = Path('mapk_replicate_cache_25')
pcoa_cache_file = None

for rep_dir in sorted(super_cache.glob('replicate_*')):
    cache_files = list(rep_dir.glob('pcoa_structures_*.pkl'))
    if cache_files:
        pcoa_cache_file = cache_files[0]
        print(f"Loading pCOA structures from: {pcoa_cache_file}")
        break

if not pcoa_cache_file:
    print("✗ No PCOA cache found in mapk_replicate_cache!")
else:
    with open(pcoa_cache_file, 'rb') as f:
        pcoa_structures = pickle.load(f)
    
    # Define variables (need this for visualization)
    VARIABLES = ['ERK', 'MEK', 'cFos', 'Fra1', 'cJun', 'CyclinD', 'Bcl2']
    INHIBITIONS = {'MEK': 'I_MEK', 'ERK': 'I_ERK', 'CyclinD': 'I_CyclinD'}
    EXOGENOUS = 'GF'
    
    for form in ['mass_action', 'mm', 'hill']:
        if form not in pcoa_structures:
            continue
        
        structure, _, _, _ = pcoa_structures[form]
        
        # Build graph
        G = nx.DiGraph()
        for var in structure.keys():
            G.add_node(var)
        for target, parents in structure.items():
            for parent in parents:
                G.add_edge(parent, target)
        
        # Layout with tighter spacing
        try:
            layers = list(nx.topological_generations(G))
            pos = {}
            for layer_idx, layer_nodes in enumerate(layers):
                layer_nodes = sorted(layer_nodes)
                n = len(layer_nodes)
                for i, node in enumerate(layer_nodes):
                    x = layer_idx * 16.0  # Reduced from 20.0
                    y = (i - (n - 1) / 2) * 12.0  # Reduced from 15.0
                    pos[node] = (x, y)
            
            if 'Fra1' in pos and pos['Fra1'][0] < 10:
                pos['Fra1'] = (10, -24)  # Adjusted
        except:
            pos = nx.spring_layout(G, k=4, iterations=100, seed=42)
        
        # Draw with enhanced parameters
        fig, ax = plt.subplots(figsize=(24, 16))
        
        external = ({EXOGENOUS} | set(INHIBITIONS.values())) & set(G.nodes())
        original = {v for v in G.nodes() if v in VARIABLES}
        R_vars = {v for v in G.nodes() if v.startswith('R_')}
        
        # Enhanced edges - thicker lines, larger arrows, curved
        nx.draw_networkx_edges(G, pos, ax=ax, edge_color='black',
                              arrowsize=70, arrowstyle='->', width=8,
                              min_source_margin=80, min_target_margin=80,
                              connectionstyle='arc3,rad=0.2')
        
        # Enhanced nodes - larger, thicker borders (inhibitions get extra space)
        if external:
            nx.draw_networkx_nodes(G, pos, nodelist=list(external),
                                  node_color='#FFD966', node_shape='s', node_size=24000,
                                  edgecolors='black', linewidths=8, ax=ax)
        
        if original:
            nx.draw_networkx_nodes(G, pos, nodelist=list(original),
                                  node_color='#6FA8DC', node_shape='o', node_size=18000,
                                  edgecolors='black', linewidths=8, ax=ax)
        
        if R_vars:
            nx.draw_networkx_nodes(G, pos, nodelist=list(R_vars),
                                  node_color='#8FBC8F', node_shape='o', node_size=14000,
                                  edgecolors='black', linewidths=7, ax=ax)
        
        # Enhanced labels - larger font
        all_labels = {}
        for node in G.nodes():
            if node.startswith('R_'):
                all_labels[node] = node.replace('R_', '').replace('_', '/')
            else:
                all_labels[node] = node
        
        nx.draw_networkx_labels(G, pos, labels=all_labels, font_size=28, font_weight='bold', ax=ax)
        
        ax.axis('off')
        ax.margins(0.08)  # Reduced from 0.3
        plt.tight_layout(pad=0.1)
        
        plt.savefig(f'figures/pcoa_structure_{form}.png', dpi=300, bbox_inches='tight', 
                   facecolor='white', pad_inches=0.1)
        print(f"✓ {form} pCOA structure saved")
        plt.close()
        
        # ============================================================================
        # CLEAN VERSION for MA (all inhibitions, no R variables)
        # ============================================================================
        if form == 'mass_action':
            clean_struct = {}
            R_to_parents = {v: p for v, p in structure.items() if v.startswith('R_')}
            
            # Keep ALL inhibitions (I_MEK, I_ERK, I_CyclinD)
            VALID_INHIBITIONS = {'GF', 'I_MEK', 'I_ERK', 'I_CyclinD'}
            
            for var, parents in structure.items():
                # Skip R variables themselves
                if var.startswith('R_'):
                    continue
                
                if var.startswith('I_') and var not in VALID_INHIBITIONS:
                    continue
                
                clean_p = []
                for p in parents:
                    # Skip other inhibitions
                    if p.startswith('I_') and p not in VALID_INHIBITIONS:
                        continue
                    
                    # Expand R variables to their parents
                    if p.startswith('R_') and p in R_to_parents:
                        expanded = [gp for gp in R_to_parents[p] 
                                   if not gp.startswith('I_') or gp in VALID_INHIBITIONS]
                        clean_p.extend(expanded)
                    else:
                        clean_p.append(p)
                
                if clean_p:
                    clean_struct[var] = list(set(clean_p))
            
            # Build clean graph
            G_clean = nx.DiGraph()
            for var in clean_struct.keys():
                G_clean.add_node(var)
            for target, parents in clean_struct.items():
                for parent in parents:
                    G_clean.add_edge(parent, target)
            
            # Layout with tighter spacing
            try:
                layers = list(nx.topological_generations(G_clean))
                pos = {}
                for layer_idx, layer_nodes in enumerate(layers):
                    layer_nodes = sorted(layer_nodes)
                    n = len(layer_nodes)
                    for i, node in enumerate(layer_nodes):
                        x = layer_idx * 16.0  # Reduced from 20.0
                        y = (i - (n - 1) / 2) * 12.0  # Reduced from 15.0
                        pos[node] = (x, y)
            except:
                pos = nx.spring_layout(G_clean, k=4, iterations=100, seed=42)
            
            # Draw with enhanced parameters
            fig, ax = plt.subplots(figsize=(24, 16))
            
            external = VALID_INHIBITIONS & set(G_clean.nodes())
            original = {v for v in G_clean.nodes() if v in VARIABLES}
            
            # Enhanced edges - curved
            nx.draw_networkx_edges(G_clean, pos, ax=ax, edge_color='black',
                                  arrowsize=70, arrowstyle='->', width=8,
                                  min_source_margin=80, min_target_margin=80,
                                  connectionstyle='arc3,rad=0.2')
            
            # Enhanced nodes - inhibitions get extra space
            if external:
                nx.draw_networkx_nodes(G_clean, pos, nodelist=list(external),
                                      node_color='#FFD966', node_shape='s', node_size=24000,
                                      edgecolors='black', linewidths=8, ax=ax)
            
            if original:
                nx.draw_networkx_nodes(G_clean, pos, nodelist=list(original),
                                      node_color='#6FA8DC', node_shape='o', node_size=18000,
                                      edgecolors='black', linewidths=8, ax=ax)
            
            # Enhanced labels
            nx.draw_networkx_labels(G_clean, pos, font_size=28, font_weight='bold', ax=ax)
            
            ax.axis('off')
            ax.margins(0.08)  # Reduced from 0.3
            plt.tight_layout(pad=0.1)
            
            plt.savefig(f'figures/pcoa_structure_mass_action_clean.png', dpi=300, 
                       bbox_inches='tight', facecolor='white', pad_inches=0.1)
            print(f"✓ MA clean (all inhibitions, no R vars) saved")
            plt.close()

print("\n✓ ALL DONE")

✓ PKN saved
Loading pCOA structures from: mapk_replicate_cache_25/replicate_00/pcoa_structures_81e04e647bc3.pkl
✓ mass_action pCOA structure saved
✓ MA clean (all inhibitions, no R vars) saved
✓ mm pCOA structure saved
✓ hill pCOA structure saved

✓ ALL DONE
